# AI in Capacity Management Demo

Team - Siddhant Sahu

In [ ]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import seaborn as sns

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

from search import parse_demand, get_weight
from retrieval import Retrieval

### Input 1 - Demand Data

Demand data stored in a csv file.

In [ ]:
demand_data = pd.read_csv("../data/demand.csv", sep="\t").fillna("")
demand_data

### Input 2 - Weights for Search Factors

Weight factors also stored in a csv file.

In [ ]:
weights_data = pd.read_csv("../data/weights.csv", sep="\t")
weights_data

### Interactive Demo

Check results when different combinations of demand + weight data are selected.

In [ ]:
# select the dataset here
dataset = "sample"  # or "sample"

if dataset == "mock":
    emp_meta_csv = "../data/emp_meta_mock.csv"
    emp_skills_csv = "../data/emp_skills_mock.csv"
elif dataset == "sample":
    emp_meta_csv = "../data/emp_meta.csv"
    emp_skills_csv = "../data/emp_skills.csv"

obj = Retrieval(emp_skills_csv, emp_meta_csv)

In [ ]:
@interact
def show_search_results(
    requestor=[f"Req_{i}" for i in range(1, 8)],
    criteria=[f"Criteria{i}" for i in range(1, 4)],
    sort_by_dept=False,
):
    demand_idx = int(requestor.replace("Req_", "")) - 1
    weight_idx = int(criteria.replace("Criteria", "")) - 1
    demand = parse_demand("../data/demand.csv", demand_idx)
    weight = get_weight("../data/weights.csv", weight_idx)
    res = obj.get_results(demand, weight, sort_by_dept).rename(
        columns={"years_of_experience": "experience"}
    )
    cm = sns.light_palette("green", as_cmap=True)
    return (
        res.style.background_gradient(cmap=cm, subset=["fitment"])
        .format(
            {
                "fitment": "{:.1%}",
                "technical": "{:.0f}",
                "functional": "{:.0f}",
                "process": "{:.0f}",
            }
        )
        .hide_index()
        .hide_columns("dept_similarity")
    )

### Explanation

* When `sort_by_dept` is checked, the results will be sorted by department first, fitment scores next.
* When unchecked, the results will be sorted by fitment scores, without considering department of the employee.